# AAI 594 — Assignment 3

## Building Agent Tools

**In this lab you will:**
- **Required (Sections 1–5):** Create **Unity Catalog function tools** — one SQL function and one Python function — and test them.
- **Required (Section 6):** Set up **semantic search with embeddings** on the UltraFeedback dataset so an agent can find similar instructions by meaning.
- **Required (Section 7):** Configure an **external MCP server** (You.com web search) in Cursor so your agent can access live web information.
- **Optional, strongly encouraged (Section 8):** Create an **Agent Skill** (`SKILL.md`) that documents the tools you built.

### The big picture

Over Weeks 3–5 you are building an **UltraFeedback Expert** agent — an AI assistant that helps users explore and understand LLM preference data. This week you create the **tools**; next week you wire them into a working agent; in Week 5 you evaluate how well it performs.

| Week | What you do | Deliverable |
|------|------------|-------------|
| 3 (this week) | Build tools: UC functions, semantic search, MCP | Tested tools + MCP config |
| 4 | Wire tools into an agent; register a prompt; compare LLMs | Working agent |
| 5 | Evaluate the agent with judges and an eval dataset | Evaluation report |

**Readings this week:**
- [Practical Guide for Agentic AI Workflows](https://arxiv.org/pdf/2512.08769)
- [MCP Architecture](https://modelcontextprotocol.io/docs/learn/architecture)

**Key docs:**
- [Create AI agent tools with UC functions](https://docs.databricks.com/aws/en/generative-ai/agent-framework/create-custom-tool)
- [Foundation Model APIs — Embeddings](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html)
- [You.com MCP Server](https://docs.you.com/developer-resources/mcp-server)

---
## 1. Why agents need tools *(Required)*

An LLM on its own can only generate text. **Tools** give agents the ability to *act* — query databases, search the web, look up facts, run computations. In this assignment you'll create three kinds of tools:

| Tool type | What it does | Example |
|-----------|-------------|--------|
| **UC SQL function** | Deterministic lookup against structured data | "How many rows come from `evol_instruct`?" |
| **UC Python function** | Custom computation or text processing | "Analyze the complexity of this instruction" |
| **Semantic search** | Embedding-based similarity search over text | "Find instructions similar to *Explain quantum tunneling*" |
| **External MCP** | Access external services (web search, APIs) | "Search the web for recent LLM benchmarks" |

Each tool is registered in a place the agent can discover it — Unity Catalog for functions and semantic search, MCP for external services.

---
## 2. Install dependencies *(Required)*

We need two packages:
- `unitycatalog-ai[databricks]` — the Unity Catalog AI client for creating and testing UC functions as agent tools.
- `numpy` — for computing cosine similarity between embedding vectors.

**Docs:** [Unity Catalog AI](https://docs.unitycatalog.io/ai/) · [Foundation Model APIs](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html)

In [ ]:
# Install the UC AI client (for creating/testing UC functions as tools)
# and numpy (for computing cosine similarity in semantic search)
%pip install unitycatalog-ai[databricks] numpy
dbutils.library.restartPython()

---
## 3. Verify your data *(Required)*

Confirm the UltraFeedback table from Assignment 1 is still available. If you get an error, re-run Assignment 1 first.

In [ ]:
# Quick check: confirm the table exists, show schema and row count
df = spark.table("main.default.assignment_file")
print(f"Row count: {df.count():,}")
print(f"Columns:  {df.columns}")
df.printSchema()
display(df.limit(3))

---
## 4. Create Unity Catalog function tools *(Required)*

Unity Catalog functions are UDFs registered in `catalog.schema.function_name`. When an agent needs a tool, it calls the function by name. Two patterns are common:

1. **SQL functions** — best for deterministic lookups against tables (e.g., counts, filters, joins).
2. **Python functions** — best for custom logic, text processing, or computations that don't map cleanly to SQL.

You'll create two SQL functions, a Python function, then build your own.

These functions exist in Unity Catalog, so you can see them in the UI after registration

**Docs:** [Create AI agent tools with UC functions](https://docs.databricks.com/aws/en/generative-ai/agent-framework/create-custom-tool) · [CREATE FUNCTION syntax](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-create-sql-function)

### 4.1 UC SQL functions

#### all_model_combinations

This function shows all combinations of chosen & rejected models, sorted by the most common combinations.

**Key points:**
- The `COMMENT` on the function and its parameters helps the agent understand *when* and *how* to use the tool. Write clear, descriptive comments.
- The function returns a `TABLE` which has all the data in the query returned

In [ ]:
%sql
CREATE OR REPLACE FUNCTION main.default.all_model_combinations()
RETURNS TABLE
LANGUAGE SQL
COMMENT 'Shows all combinations of chosen and rejected models, counts and average ratings'
RETURN (
  SELECT 
    `chosen-model`,
    `rejected-model`,
    count(source) as records,
    avg(`chosen-rating`) as avg_chosen_rating,
    avg(`rejected-rating`) as avg_rejected_rating
  FROM main.default.assignment_file
  GROUP BY 1,2
  ORDER BY 3 desc
)

In [ ]:
%sql
select * from main.default.all_model_combinations() limit 10

#### compare_models

The function below compares two specific models to see how frequently an individual model wins vs. loses.

This also returns a table with the count of each scenario, as well as the average winning and losing rating

In [ ]:
%sql
CREATE OR REPLACE FUNCTION main.default.compare_models(
  model_a STRING COMMENT 'First model name to compare e.g. gpt-4',
  model_b STRING COMMENT 'Second model name to compare'
)
RETURNS TABLE
LANGUAGE SQL
COMMENT 'Compares two models by how often each was chosen vs rejected in the UltraFeedback dataset. Returns a summary string with win counts for each model.'
RETURN (
  SELECT 
    CASE when `chosen-model` = model_a AND `rejected-model` = model_b THEN 'A_win'
         when `chosen-model` = model_b AND `rejected-model` = model_a THEN 'B_win'
         else 'other' end as comparison_scenario,
    count(*) as win_count,
    avg(`chosen-rating`) as avg_chosen_rating,
    avg(`rejected-rating`) as avg_rejected_rating
  FROM main.default.assignment_file
  WHERE `chosen-model` IN (model_a, model_b)
     AND `rejected-model` IN (model_a, model_b)
  GROUP BY 1
  ORDER BY 1
)

In [ ]:
%sql
select * 
from main.default.compare_models('gpt-3.5-turbo', 'alpaca-7b')

### 4.2 Python function: `analyze_instruction`

This function takes an instruction text and returns complexity metrics (word count, sentence count, estimated complexity level). An agent could use this to assess how complex a prompt is before deciding how to handle it.

**Key points:**
- Python UC functions must have **type hints** on all arguments and the return value.
- **Imports go inside the function body** — they won't be resolved otherwise.
- Use [Google-style docstrings](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods) so the agent can parse the description.

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

# Initialize the Databricks Function Client
uc_client = DatabricksFunctionClient()

# Define the Python function with type hints and a clear docstring.
# NOTE: all imports must be INSIDE the function body.
def analyze_instruction(instruction: str) -> str:
    """
    Analyzes the complexity and characteristics of an instruction prompt.

    Returns word count, sentence count, average word length, estimated
    complexity level (low/medium/high), and whether the text is a question.
    Use this to assess instruction difficulty before generating a response.

    Args:
        instruction: The instruction or prompt text to analyze.

    Returns:
        A JSON string with analysis metrics.
    """
    import json
    import re

    words = instruction.split()
    word_count = len(words)
    sentences = [s.strip() for s in re.split(r'[.!?]+', instruction) if s.strip()]
    sentence_count = len(sentences)
    avg_word_length = round(sum(len(w) for w in words) / max(word_count, 1), 1)
    is_question = instruction.strip().endswith('?')

    if word_count > 50 or sentence_count > 3:
        complexity = "high"
    elif word_count > 20:
        complexity = "medium"
    else:
        complexity = "low"

    return json.dumps({
        "word_count": word_count,
        "sentence_count": sentence_count,
        "avg_word_length": avg_word_length,
        "complexity": complexity,
        "is_question": is_question
    })

# Register the function in Unity Catalog (main.default schema)
function_info = uc_client.create_python_function(
    func=analyze_instruction,
    catalog="main",
    schema="default",
    replace=True  # overwrite if it already exists
)
print(f"Registered: {function_info.full_name}")

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

uc_client = DatabricksFunctionClient()

# Test the function out that we just created

result = uc_client.execute_function(
    function_name="main.default.analyze_instruction",
    parameters={"instruction": "Heisenberg explained resonance theory, which is a structure-activity relationship that describes the effect of complementary chemical groups on a drug's properties and actions (such as solubility, absorption, and therapeutic effects). These complementary groups can be used to reduce toxicity and increase the potency of drug compounds"})

print(result.value)

%md
### 4.3 Your turn: create a function *(Required)*

Create **two additional UC function** (SQL or Python or both) that would be useful for the UltraFeedback Expert agent. Some  ideas:

| Idea | Type | What it does |x
|------|------|--------------|
| `count_model_appearances` | SQL | Count how often a model appears as chosen vs. rejected |
| `get_sample_pairs` | SQL | Return N example chosen/rejected pairs for a given source |
| `format_comparison` | Python | Take a chosen and rejected response and format them side-by-side |
| `extract_keywords` | Python | Pull key terms from an instruction for categorization |

Make sure your function has:
- A clear `COMMENT` (SQL) or docstring (Python) explaining what it does and when to use it
- Type hints (Python) or typed parameters (SQL)
- A test cell showing it works just like the examples above

In [ ]:
# CREATE YOUR FUNCTIONS HERE
# Use either %%sql for a SQL function or Python with uc_client.create_python_function()
# Then add a test cell below to verify it works.

In [ ]:
# TEST YOUR FUNCTIONS HERE

---
## 5. List your registered tools

Before moving on, verify all your UC functions are registered. The cell below lists functions in `main.default`.

In [ ]:
%%sql
-- List all functions you've created in main.default
SHOW USER FUNCTIONS IN main.default;

## 6. Semantic Search with Embeddings *(Required)*

Semantic search lets an agent find **similar text by meaning** — not just exact keyword matches. For the UltraFeedback Expert, this means the agent can find instructions similar to a user's question, even if the wording is different.

**How it works:**
1. **Embed** each instruction using a Foundation Model embedding endpoint (`databricks-gte-large-en`).
2. **Store** the embeddings in a Delta table.
3. At query time, embed the user's question and compute **cosine similarity** against all stored embeddings.

This approach uses the Foundation Model API (available on Free Edition) instead of Vector Search endpoints (which require a quota not available on Free Edition).

> **Why not Vector Search?** Databricks Vector Search requires provisioned endpoints that exceed the Free Edition quota. The manual embedding approach teaches the same concepts (embeddings, similarity search, retrieval) and works with no extra resources.

**Docs:** [Foundation Model APIs — Embeddings](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html)

### 6.1 Prepare a source table

We'll create a focused table with 500 unique instructions. This keeps embedding costs low while giving the agent plenty of material to search.

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

# Create a table of 500 unique instructions for semantic search
try:
    spark.table("main.default.ultrafeedback_embeddings").limit(1)
    print("Embeddings table already exists.")
except:
    print("Creating source table with 500 unique instructions...")
    source_df = (
        spark.table("main.default.assignment_file")
        .select("source", "instruction")
        .dropDuplicates(["instruction"])
        .limit(500)
        .withColumn("id", monotonically_increasing_id())
    )
    source_df.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("main.default.ultrafeedback_embeddings")
    print(f"Created table with {source_df.count()} rows.")

display(spark.table("main.default.ultrafeedback_embeddings").limit(3))

### 6.2 Compute embeddings

We'll use the `databricks-gte-large-en` Foundation Model endpoint to compute embeddings. The endpoint accepts batches of text and returns vectors (1024 dimensions for GTE-Large).

We embed in batches because the API has input size limits.

In [ ]:
import mlflow.deployments
import json

client = mlflow.deployments.get_deploy_client("databricks")

def get_embeddings_batch(texts, endpoint="databricks-gte-large-en"):
    """Embed a list of texts using the Foundation Model API."""
    response = client.predict(
        endpoint=endpoint,
        inputs={"input": texts}
    )
    return [item["embedding"] for item in response["data"]]

# Load instructions from the source table
instructions_df = spark.table("main.default.ultrafeedback_embeddings").toPandas()
instructions = instructions_df["instruction"].tolist()
ids = instructions_df["id"].tolist()

print(f"Embedding {len(instructions)} instructions...")

# Embed in batches of 20
BATCH_SIZE = 20
all_embeddings = []
for i in range(0, len(instructions), BATCH_SIZE):
    batch = instructions[i:i + BATCH_SIZE]
    batch_embeddings = get_embeddings_batch(batch)
    all_embeddings.extend(batch_embeddings)
    if (i // BATCH_SIZE) % 5 == 0:
        print(f"  Embedded {min(i + BATCH_SIZE, len(instructions))}/{len(instructions)}")

print(f"Done. Each embedding has {len(all_embeddings[0])} dimensions.")

In [ ]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType, FloatType

# Build a DataFrame with id, instruction, source, and embedding
embedding_records = []
for idx, (row_id, instr, emb) in enumerate(zip(ids, instructions, all_embeddings)):
    embedding_records.append({
        "id": int(row_id),
        "instruction": instr,
        "source": instructions_df.iloc[idx]["source"],
        "embedding": emb
    })

schema = StructType([
    StructField("id", LongType(), False),
    StructField("instruction", StringType(), False),
    StructField("source", StringType(), True),
    StructField("embedding", ArrayType(FloatType()), False),
])

emb_spark_df = spark.createDataFrame(embedding_records, schema=schema)

# Save to Delta — this is our "vector index"
emb_spark_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.default.ultrafeedback_embeddings")

print(f"Saved {len(embedding_records)} embeddings to main.default.ultrafeedback_embeddings.")
display(spark.table("main.default.ultrafeedback_embeddings").select("id", "instruction", "source").limit(3))

### 6.3 Create a similarity search function

Now we create a Python UC function that the agent can call. Given a query, it embeds the query, loads the stored embeddings, computes cosine similarity, and returns the top matches.

In [ ]:
def search_similar_instructions(query: str, top_k: int = 3) -> str:
    """
    Finds instructions in the UltraFeedback dataset that are semantically similar to the query.
    Uses embedding-based cosine similarity search. Returns the top_k most similar instructions
    with their similarity scores and source names as a JSON string.
    Use this when a user wants to find instructions similar to a given topic or phrase.
    """
    import json
    import mlflow.deployments

    deploy_client = mlflow.deployments.get_deploy_client("databricks")

    # Embed the query
    response = deploy_client.predict(
        endpoint="databricks-gte-large-en",
        inputs={"input": [query]}
    )
    query_emb = response["data"][0]["embedding"]

    # Load stored embeddings from the Delta table
    from pyspark.sql import SparkSession
    spark_session = SparkSession.builder.getOrCreate()
    emb_df = spark_session.table("main.default.ultrafeedback_embeddings").toPandas()

    # Compute cosine similarity
    import numpy as np
    query_vec = np.array(query_emb)
    query_norm = np.linalg.norm(query_vec)

    similarities = []
    for _, row in emb_df.iterrows():
        doc_vec = np.array(row["embedding"])
        doc_norm = np.linalg.norm(doc_vec)
        if query_norm == 0 or doc_norm == 0:
            sim = 0.0
        else:
            sim = float(np.dot(query_vec, doc_vec) / (query_norm * doc_norm))
        similarities.append({
            "instruction": row["instruction"],
            "source": row["source"],
            "similarity": round(sim, 4)
        })

    # Sort by similarity and return top_k
    similarities.sort(key=lambda x: x["similarity"], reverse=True)
    top_results = similarities[:top_k]

    return json.dumps({
        "query": query,
        "num_results": len(top_results),
        "results": top_results
    })

function_info = uc_client.create_python_function(
    func=search_similar_instructions,
    catalog="main",
    schema="default",
    replace=True
)
print(f"Registered: {function_info.full_name}")

### 6.4 Test the similarity search

In [ ]:
# Test similarity search
import json

test_queries = [
    "Explain quantum computing",
    "Write a Python function",
    "What are the health benefits of exercise?",
]

for query in test_queries:
    result = uc_client.execute_function(
        function_name="main.default.search_similar_instructions",
        parameters={"query": query, "top_k": 3}
    )
    parsed = json.loads(result.value)
    print(f"\nQuery: {query}")
    for r in parsed["results"]:
        print(f"  [{r['similarity']:.3f}] ({r['source']}) {r['instruction'][:80]}...")

---
## 7. Configure an external MCP server *(Required)*

The **Model Context Protocol (MCP)** is an open standard that lets AI assistants connect to external tools and data sources. By adding an MCP server to Cursor, your agent gains access to live capabilities — in this case, **web search**.

You'll configure the **You.com MCP server**, which provides:
- `you-search` — web and news search with filtering
- `you-contents` — extract content from URLs in markdown format

This means your agent will be able to search the web for current information about LLMs, benchmarks, and research papers — something it can't do with just the UltraFeedback dataset.

**Docs:** [You.com MCP Server](https://docs.you.com/developer-resources/mcp-server) · [MCP in Cursor](https://cursor.com/docs/context/mcp) · [MCP Architecture](https://modelcontextprotocol.io/docs/learn/architecture)

### 7.1 Get a You.com API key

1. Go to [you.com/platform](https://you.com/platform).
2. Sign in or create an account.
3. Generate an API key and copy it. **Keep it safe — you'll need it in the next step.**

### 7.2 Add the MCP server to Cursor

Cursor reads MCP server configuration from a JSON file. You can configure it at the **project level** (only this project) or **globally** (all projects).

#### Option A: Project-level (recommended for this course)

Create or edit `.cursor/mcp.json` in your project root:

```json
{
  "mcpServers": {
    "ydc-server": {
      "url": "https://api.you.com/mcp",
      "headers": {
        "Authorization": "Bearer <YOUR-YOU-COM-API-KEY>"
      }
    }
  }
}
```

Replace `<YOUR-YOU-COM-API-KEY>` with your actual API key.

#### Option B: Global

Edit `~/.cursor/mcp.json` to make this available across all your Cursor projects.

#### One-click install

Alternatively, you can install directly from Cursor's MCP directory: visit the [You.com MCP page](https://docs.you.com/developer-resources/mcp-server) and click the **"Install MCP Server"** button for Cursor.

> **Tip:** After saving `mcp.json`, restart Cursor or reload the window (`Cmd+Shift+P` → "Reload Window"). You should see the You.com tools available in the Agent chat.

### 7.3 Test your MCP connection

Open Cursor's **Agent chat** (not the regular chat) and try a query that requires live web search:

- *"Search the web for the latest LLM benchmarks from 2025-2026."*
- *"What are the top open-source LLMs released in the last 6 months?"*

The agent should use the `you-search` tool to fetch live results. You'll see a tool-call indicator in the chat.

> **Take a screenshot** of the agent using the You.com MCP tool in Cursor. Include it in your submission as `screenshots/mcp_you_com.png`.

> **Troubleshooting:**
> - If the tools don't appear, check that `mcp.json` is valid JSON (no trailing commas, correct quoting).
> - Verify your API key is active at [you.com/platform](https://you.com/platform).
> - Try restarting Cursor after editing the config.
> - Go to Cursor Settings → Agents tab and turn off Cursor's built-in web search to avoid conflicts.

### 7.4 Alternative MCP servers

You.com is the recommended MCP for this assignment, but you're welcome to configure additional servers. Here are some useful options:

| MCP Server | What it provides | Get started |
|------------|-----------------|-------------|
| **You.com** (required) | Web search, news, content extraction | [you.com/platform](https://you.com/platform) |
| **Brave Search** | Privacy-focused web search | [brave.com/search/api](https://brave.com/search/api/) |
| **Tavily** | AI-optimized search for agents | [tavily.com](https://tavily.com/) |
| **GitHub** | Code search, issues, PRs | [github.com/github/github-mcp-server](https://github.com/github/github-mcp-server) |
| **Filesystem** | Read/write local files | Built into many MCP clients |

Each follows the same pattern: get an API key, add a server entry to `mcp.json`, restart Cursor. The [MCP Registry](https://registry.modelcontextprotocol.io/) has a full catalog of available servers.

---
## 8. Bonus: Create an Agent Skill *(Optional, strongly encouraged)*

An **Agent Skill** is a markdown document (`SKILL.md`) that gives an AI assistant domain knowledge. When a skill is loaded, the assistant knows how to use specific tools, follow procedures, and avoid common mistakes — without you having to explain everything in every prompt.

Think of it as a **user manual for your agent's tools**, written so another AI can follow it.

### Why this matters

You've just built several tools (UC functions, semantic search, MCP). But an AI assistant doesn't automatically know *when* to use each one, *how* to call them, or *what to watch out for*. A skill bridges that gap.

### Create a skill

Create a file called `SKILL.md` in your `assignment_3/` folder with the following structure:

```markdown
---
name: ultrafeedback-expert
description: >
  Tools and knowledge for exploring the UltraFeedback LLM preference dataset.
  Activate when: user asks about LLM preferences, model comparisons, or
  instruction quality in the UltraFeedback dataset.
---

# UltraFeedback Expert

## When to Use This Skill

**Trigger patterns:**
- "UltraFeedback" or "preference data" or "chosen vs rejected"
- "Which model is preferred" or "model comparison"
- "Find similar instructions" or "semantic search"

## Available Tools

| Tool | Type | What it does |
|------|------|--------------|
| `main.default.lookup_source_info` | UC SQL | Returns row count and sample for a source |
| `main.default.analyze_instruction` | UC Python | Analyzes instruction complexity |
| `main.default.compare_models` | UC SQL | Compares two models by chosen vs rejected counts |
| `main.default.get_model_win_rate` | UC SQL | Returns a model's win rate in the dataset |
| `main.default.classify_instruction_topic` | UC Python | Classifies instruction topic by keywords |
| `main.default.search_similar_instructions` | UC Python | Embedding-based semantic search over 500 instructions |
| You.com MCP | External MCP | Live web search for current LLM info |

## Procedures

### Answering "What sources are in the dataset?"
1. Call `lookup_source_info` for each known source.
2. Summarize counts and sample instructions.

### Finding similar instructions
1. Call `search_similar_instructions` with the user's text.
2. Return the top 3-5 matches with their sources and similarity scores.

## Gotchas
- Embeddings table (`main.default.ultrafeedback_embeddings`) must exist with pre-computed vectors.
- Column names with hyphens (e.g., `chosen-model`) need backtick escaping.
```

Fill in the details based on the actual tools you created. You can use this skill in Cursor by placing it in `~/.cursor/skills/` or referencing it in a project rule.

**Docs:** [Agent Skills standard](https://github.com/xnano-ai/agentskills) · [Cursor Rules](https://cursor.com/docs/context/rules)

---
## Lab complete

### Required (Sections 1–7)
- [ ] **Section 3:** Verified the UltraFeedback table exists.
- [ ] **Section 4:** Created and tested the SQL function (`lookup_source_info`) and Python function (`analyze_instruction`).
- [ ] **Section 4.3:** Created and tested your own UC function.
- [ ] **Section 5:** Listed all registered UC functions.
- [ ] **Section 6:** Created embeddings for 500 instructions, registered the  UC function, and tested semantic search.
- [ ] **Section 7:** Configured the You.com MCP server in Cursor and tested it (screenshot taken).

### Optional but strongly encouraged (Section 8)
- [ ] **Section 8:** Created a `SKILL.md` documenting your tools.

**Submit:** Your executed notebook (`.ipynb` with all outputs) and the completed `SUBMISSION_3.md`. Include screenshots in the `screenshots/` folder.

*Next week you'll wire these tools into a working agent, register a prompt, and compare different LLMs.*